In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import models, layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
#from pyDeepInsight import ImageTransformer, LogScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv(r'C:\Users\anive\Documents\GitHub\Traffic-classification-and-mitigation-of-DDoS-Attack-in-SDN-\sdn_traffic_dataset.csv')
df.shape

(104345, 23)

In [4]:
df.size

2399935

In [31]:
df.sample(100)

,pktcount,bytecount,dur,dur_nsec,tot_dur,flows,packetins,pktperflow,byteperflow,pktrate,...,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label,Protocol_ICMP,Protocol_TCP,Protocol_UDP
97080,10,980,11,480000000,1.148000e+10,11,1286,0,0,0,...,3,77038,73494,0,0.0,0.0,0,1,0,0
63375,9,882,9,934000000,9.934000e+09,5,1065,0,0,0,...,3,78624,135538498,1,1.0,2.0,0,1,0,0
82557,54720,2954880,97,584000000,9.758400e+10,5,16540,17365,937710,578,...,3,6437513,147084102,0,0.0,0.0,1,0,1,0
6950,32713,34872058,71,684000000,7.168400e+10,4,1931,13385,14268410,446,...,2,3472,1172,0,0.0,0.0,0,0,0,1
16843,48049,50067058,153,436000000,1.530000e+11,3,1931,9405,9800010,313,...,2,3599,1492,0,0.0,0.0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86293,595,58310,613,203000000,6.130000e+11,3,2997,29,2842,0,...,3,135525688,65982,0,0.0,0.0,0,1,0,0
45115,137588,7429752,240,104000000,2.400000e+11,5,18351,14915,805410,497,...,3,466327121,320775026,428,230.0,658.0,0,0,1,0
70852,26132,1724888,81,133000000,8.113300e+10,5,12106,8968,592008,298,...,4,289016214,11441441,4148,158.0,4306.0,1,0,1,0
4047,100312,106932592,222,908000000,2.230000e+11,4,1931,13308,14186328,443,...,2,3649,1242,0,0.0,0.0,0,0,0,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104345 entries, 0 to 104344
Data columns (total 23 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   dt           104345 non-null  int64  
 1   switch       104345 non-null  int64  
 2   src          104345 non-null  object 
 3   dst          104345 non-null  object 
 4   pktcount     104345 non-null  int64  
 5   bytecount    104345 non-null  int64  
 6   dur          104345 non-null  int64  
 7   dur_nsec     104345 non-null  int64  
 8   tot_dur      104345 non-null  float64
 9   flows        104345 non-null  int64  
 10  packetins    104345 non-null  int64  
 11  pktperflow   104345 non-null  int64  
 12  byteperflow  104345 non-null  int64  
 13  pktrate      104345 non-null  int64  
 14  Pairflow     104345 non-null  int64  
 15  Protocol     104345 non-null  object 
 16  port_no      104345 non-null  int64  
 17  tx_bytes     104345 non-null  int64  
 18  rx_bytes     104345 non-

In [7]:
df.isna().sum()

dt               0
switch           0
src              0
dst              0
pktcount         0
bytecount        0
dur              0
dur_nsec         0
tot_dur          0
flows            0
packetins        0
pktperflow       0
byteperflow      0
pktrate          0
Pairflow         0
Protocol         0
port_no          0
tx_bytes         0
rx_bytes         0
tx_kbps          0
rx_kbps        506
tot_kbps       506
label            0
dtype: int64

In [8]:
df.head()

,dt,switch,src,dst,pktcount,bytecount,dur,dur_nsec,tot_dur,flows,...,pktrate,Pairflow,Protocol,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label
0,11425,1,10.0.0.1,10.0.0.8,45304,48294064,100,716000000,1.010000e+11,3,...,451,0,UDP,3,143928631,3917,0,0.0,0.0,0
1,11605,1,10.0.0.1,10.0.0.8,126395,134737070,280,734000000,2.810000e+11,2,...,451,0,UDP,4,3842,3520,0,0.0,0.0,0
2,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,...,451,0,UDP,1,3795,1242,0,0.0,0.0,0
3,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,...,451,0,UDP,2,3688,1492,0,0.0,0.0,0
4,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,...,451,0,UDP,3,3413,3665,0,0.0,0.0,0


In [9]:
df.dropna(inplace=True)
df.isna().sum()

dt             0
switch         0
src            0
dst            0
pktcount       0
bytecount      0
dur            0
dur_nsec       0
tot_dur        0
flows          0
packetins      0
pktperflow     0
byteperflow    0
pktrate        0
Pairflow       0
Protocol       0
port_no        0
tx_bytes       0
rx_bytes       0
tx_kbps        0
rx_kbps        0
tot_kbps       0
label          0
dtype: int64

In [10]:
df.drop(['dt', 'switch', 'src', 'dst'],axis=1,inplace=True)
df.head()

,pktcount,bytecount,dur,dur_nsec,tot_dur,flows,packetins,pktperflow,byteperflow,pktrate,Pairflow,Protocol,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label
0,45304,48294064,100,716000000,1.010000e+11,3,1943,13535,14428310,451,0,UDP,3,143928631,3917,0,0.0,0.0,0
1,126395,134737070,280,734000000,2.810000e+11,2,1943,13531,14424046,451,0,UDP,4,3842,3520,0,0.0,0.0,0
2,90333,96294978,200,744000000,2.010000e+11,3,1943,13534,14427244,451,0,UDP,1,3795,1242,0,0.0,0.0,0
3,90333,96294978,200,744000000,2.010000e+11,3,1943,13534,14427244,451,0,UDP,2,3688,1492,0,0.0,0.0,0
4,90333,96294978,200,744000000,2.010000e+11,3,1943,13534,14427244,451,0,UDP,3,3413,3665,0,0.0,0.0,0


In [11]:
df.shape

(103839, 19)

In [12]:
df.Protocol.unique()

array(['UDP', 'TCP', 'ICMP'], dtype=object)

In [13]:
df.label.unique()

array([0, 1], dtype=int64)

In [15]:
@df=pd.get_dummies(df)

In [30]:
df.head()

,pktcount,bytecount,dur,dur_nsec,tot_dur,flows,packetins,pktperflow,byteperflow,pktrate,...,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label,Protocol_ICMP,Protocol_TCP,Protocol_UDP
0,45304,48294064,100,716000000,1.010000e+11,3,1943,13535,14428310,451,...,3,143928631,3917,0,0.0,0.0,0,0,0,1
1,126395,134737070,280,734000000,2.810000e+11,2,1943,13531,14424046,451,...,4,3842,3520,0,0.0,0.0,0,0,0,1
2,90333,96294978,200,744000000,2.010000e+11,3,1943,13534,14427244,451,...,1,3795,1242,0,0.0,0.0,0,0,0,1
3,90333,96294978,200,744000000,2.010000e+11,3,1943,13534,14427244,451,...,2,3688,1492,0,0.0,0.0,0,0,0,1
4,90333,96294978,200,744000000,2.010000e+11,3,1943,13534,14427244,451,...,3,3413,3665,0,0.0,0.0,0,0,0,1


In [17]:
##standardlize

In [18]:
scaler=MinMaxScaler()

In [19]:
df_std =scaler.fit_transform(df)


In [20]:
df_std.shape

(103839, 21)

In [21]:
new_df = pd.DataFrame(df_std, columns=df.columns)

In [22]:
new_df.head()

,pktcount,bytecount,dur,dur_nsec,tot_dur,flows,packetins,pktperflow,byteperflow,pktrate,...,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label,Protocol_ICMP,Protocol_TCP,Protocol_UDP
0,0.174242,0.328245,0.053163,0.716717,0.053723,0.066667,0.076883,0.962331,0.996744,0.96243,...,0.50,1.133295e-01,3.090061e-06,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.486123,0.915781,0.148857,0.734735,0.149468,0.000000,0.076883,0.962304,0.996717,0.96243,...,0.75,1.035450e-06,2.689292e-06,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.347427,0.654498,0.106326,0.744745,0.106915,0.066667,0.076883,0.962324,0.996737,0.96243,...,0.00,9.984414e-07,3.896647e-07,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.347427,0.654498,0.106326,0.744745,0.106915,0.066667,0.076883,0.962324,0.996737,0.96243,...,0.25,9.141880e-07,6.420382e-07,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.347427,0.654498,0.106326,0.744745,0.106915,0.066667,0.076883,0.962324,0.996737,0.96243,...,0.50,6.976491e-07,2.835669e-06,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [23]:
new_df.shape

(103839, 21)

In [24]:
new_df['class'] = new_df['label']
del new_df['label']

In [25]:
new_df.shape

(103839, 21)

In [26]:
###split

In [29]:
print(new_df)

        pktcount  bytecount       dur  dur_nsec   tot_dur     flows  \
0       0.174242   0.328245  0.053163  0.716717  0.053723  0.066667   
1       0.486123   0.915781  0.148857  0.734735  0.149468  0.000000   
2       0.347427   0.654498  0.106326  0.744745  0.106915  0.066667   
3       0.347427   0.654498  0.106326  0.744745  0.106915  0.066667   
4       0.347427   0.654498  0.106326  0.744745  0.106915  0.066667   
...          ...        ...       ...       ...       ...       ...   
103834  0.000304   0.000053  0.043062  0.842843  0.043533  0.200000   
103835  0.000304   0.000053  0.043062  0.842843  0.043533  0.200000   
103836  0.000119   0.000021  0.016481  0.805806  0.016918  0.200000   
103837  0.000119   0.000021  0.016481  0.805806  0.016918  0.200000   
103838  0.000119   0.000021  0.016481  0.805806  0.016918  0.200000   

        packetins  pktperflow  byteperflow   pktrate  ...  port_no  \
0        0.076883    0.962331     0.996744  0.962430  ...     0.50   
1      